# Transfer Learning
With transfer learning, we take a pre-trained model and retrain it on a task that has some overlap with the original training task. A good analogy for this is an artist who is skilled in one medium, such as painting, who wants to learn to practice in another medium, such as charcoal drawing. We can imagine that the skills they learned while painting would be very valuable in learning how to draw with charcoal.

As an example in deep learning, say we have a pre-trained model that is very good at recognizing different type of cars, and we want to train a model to recognize types of motorcycles. A lot of the learnings of the car model would likely be very useful, for instance the ability to recognize headlights and wheels.

Transfer learning is especially powerful when we don't have a large dataset.

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet', #Load weights pre-trained on ImageNet
    input_shape = (224,224,3),
    include_top=False,
)
base_model.summary()